## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


Using TensorFlow backend.


In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 
print(x_train, x_test)

(50000, 32, 32, 3)
[[[[-0.96192505 -0.91515971 -0.89957127]
   [-1.21134019 -1.16457485 -1.1801633 ]
   [-1.10222107 -1.13339796 -1.21134019]
   ...
   [ 0.58133111  0.17603151 -0.19809119]
   [ 0.48780043  0.06691239 -0.29162187]
   [ 0.42544665  0.05132394 -0.27603342]]

  [[-1.63222823 -1.56987445 -1.56987445]
   [-1.88164337 -1.88164337 -1.88164337]
   [-1.60105134 -1.7569358  -1.88164337]
   ...
   [ 0.0357355  -0.50986012 -1.02427884]
   [-0.02661829 -0.58780235 -1.10222107]
   [ 0.02014705 -0.52544856 -0.99310194]]

  [[-1.49193222 -1.50752066 -1.554286  ]
   [-1.63222823 -1.77252425 -1.88164337]
   [-1.11780951 -1.46075533 -1.7569358 ]
   ...
   [-0.04220673 -0.5722139  -1.10222107]
   [-0.01102984 -0.5722139  -1.10222107]
   [-0.18250275 -0.74368681 -1.22692863]]

  ...

  [[ 1.36075341  0.76839246 -0.38515255]
   [ 1.25163429  0.50338888 -1.3516362 ]
   [ 1.20486895  0.62809645 -1.47634377]
   ...
   [ 0.612508    0.19161996 -0.79045214]
   [-1.00869039 -1.39840154 -1.7725242

In [4]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

/Users/kuangchiuhuang/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [10]:

classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(filters=32, kernel_size=(3,3), padding='same', input_shape=(32, 32, 3), activation='relu'))#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(BatchNormalization())

'''自己決定MaxPooling2D放在哪裡'''
#classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Convolution2D(filters=64, kernel_size=(3,3), padding='same', activation='relu'))
classifier.add(BatchNormalization())

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(128,kernel_initializer ='uniform', activation = 'relu')) #output_dim=100,activation=relu

#輸出
classifier.add(Dense(10,kernel_initializer="uniform", activation= 'softmax'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=100,epochs=4)
classifier.summary()

Epoch 1/4
50000/50000 [==============================] - 125s 2ms/step - loss: 1.3791 - accuracy: 0.5247
Epoch 2/4
50000/50000 [==============================] - 129s 3ms/step - loss: 0.8614 - accuracy: 0.6974
Epoch 3/4
50000/50000 [==============================] - 129s 3ms/step - loss: 0.5238 - accuracy: 0.8165
Epoch 4/4
50000/50000 [==============================] - 130s 3ms/step - loss: 0.2624 - accuracy: 0.9090
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization_3 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
batch_normalization_4 (Batch (

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [11]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[0.07897794, 0.00314737, 0.153508  , 0.23454453, 0.22496071,
        0.08478475, 0.20281017, 0.01388231, 0.00237745, 0.00100674]],
      dtype=float32)

In [12]:
scores = classifier.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 6s 639us/step
Test loss: 1.409712205505371
Test accuracy: 0.6394000053405762
